In [ ]:
import numpy as np
from collections import defaultdict, deque
from graphs import graph2 as graph
class ProbabilisticGraphSampler:
    def __init__(self, graph):
        """
        Initialize the sampler with a graph.
        :param graph: 
            keys : random variables
            values : (parents, expression).
                      Example:
                      {
                          "z": ([], lambda: np.random.binomial(1, 0.5)),
                          "y": (["z"], lambda values: np.random.normal(-1.0 if values["z"] == 0 else 1.0, 1.0))
                      }
        """
        self.graph = graph
        self.values = {}
        self.sorted_nodes = self.topological_sort()

    def topological_sort(self):
        """
        Perform a topological sort of the graph.
        :return: A list of nodes in topological order.
        """
        # Build the dependency graph
        in_degree = defaultdict(int)  # Count of incoming edges for each node
        adj_list = defaultdict(list)  # Adjacency list for graph traversal

        for node, (parents, _) in self.graph.items():
            for parent in parents:
                in_degree[node] += 1
                adj_list[parent].append(node)

        # Collect nodes with no incoming edges
        queue = deque([node for node in self.graph if in_degree[node] == 0])
        sorted_nodes = []

        while queue:
            node = queue.popleft()
            sorted_nodes.append(node)

            # Reduce in-degree for child nodes
            for neighbor in adj_list[node]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    queue.append(neighbor)

        if len(sorted_nodes) != len(self.graph):
            print(sorted_nodes)
            raise ValueError("Graph contains a cycle!")

        return sorted_nodes

    def sample_trace(self):
        """
        Sample a single trace by evaluating all nodes in topological order.
        :return: A dictionary of sampled values for all nodes.
        """
        self.values.clear()  # Clear previous sampled values
        for node in self.sorted_nodes:
            parents, expression = self.graph[node]
            # Gather parent values
            parent_values = {p: self.values[p] for p in parents}
            # Evaluate the current node
            self.values[node] = expression(parent_values)
        return self.values




#print(graph2)
# Create the sampler
sampler = ProbabilisticGraphSampler(graph)

# Sample traces
for _ in range(5):
    trace = sampler.sample_trace()
    print(trace)



In [ ]:
# Test for loop
from ast_class import If, Constant, Sample, Let, Observe, Variable, Assign, For
from translator import Translator

prog = [
    Sample("x", ('normal', 0.0, 1.0)),
    Let("z",
        If(
            # condition: x>0.0
            ('>', Variable("x"), Constant(0.0)),
            Constant(10.0),
            ('+', Constant(-5.0), Constant(-5.0))
        ),
        # body => sample y
        Sample("y", ('normal', Variable("z"), 1.0))
    ),
    Observe(('normal', Variable("y"), 1.0), 0.5)
]

# We'll define an initial "acc" = 0
# prog = [
#     Assign("acc", Constant(0)),
#     Sample("x", ("normal", 0.0, 1.0)),
#     For("i", 3, [
#         Assign("acc", ("+", Variable("acc"), Variable("i")))
#     ]),
#     Observe(("normal", Variable("x"), 1.0), 0.5),
# ]

translator = Translator()
graph = translator.translate_program(prog)

print("\nFINAL GRAPH NODES:\n")
for name, node in graph.items():
    print(f"{name} -> {node}")

In [ ]:
# Test Importance Sampling
from ast_class import If, Constant, Sample, Let, Observe, Variable, Assign, For
from translator import Translator
from importance_sampling import ImportanceSampler
import math

# 1) Define a simple AST:
#    x ~ Normal(0,1)
#    observe x ~ Normal(0,1) with obs=0.5
# program_ast = [
#     Sample("x", ("normal", 0.0, 1.0)),
#     Observe(("normal", Variable("x"), 1.0), 0.5)
# ]

program_ast = [
        Sample("x", ("normal", 0.0, 1.0)),
        Observe(("normal", Variable("x"), 2.0), 0.5)
    ]

# 2) Translate AST -> Graph
translator = Translator()
graph = translator.translate_program(program_ast)

# 3) Instantiate ImportanceSampler
sampler = ImportanceSampler(graph)

# 4) Draw multiple samples
N = 5000
samples = []
weights = []
for _ in range(N):
    env, log_weight = sampler.sample_one()
    samples.append(env)
    weights.append(math.exp(log_weight))  # store weight in linear scale

# 5) Analyze results: e.g. compute weighted mean of x
weighted_sum = 0.0
total_weight = 0.0
for env, w in zip(samples, weights):
    weighted_sum += env["x"] * w
    total_weight += w
posterior_mean_x = weighted_sum / total_weight

print(f"Approx. posterior mean of x = {posterior_mean_x:.3f}")

In [ ]:
# Importance Sampling for model 1.2
from ast_class import Sample, Observe, Variable
from translator import Translator
from importance_sampling import ImportanceSampler
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats  # for the Normal PDF



mu0 = 50        # example prior mean
tau2 = 10.0       # example prior variance
y_obs = 45   # observed count


"""
(let [theta (sample (normal mu0 tau2))]
  (observe (poisson theta) y_obs)
  theta)
"""

program_ast = [
    Sample("theta", ("normal", mu0, tau2)),
    Observe(("poisson", Variable("theta")), y_obs)
]

translator = Translator()
graph = translator.translate_program(program_ast)
sampler = ImportanceSampler(graph)

N = 5000
samples = []
weights = []
for i in range(N):
    env, log_weight = sampler.sample_one()
    samples.append(env)
    weights.append(math.exp(log_weight))  # convert log weight to linear scale

weighted_sum = 0.0
total_weight = 0.0

for env, w in zip(samples, weights):
    weighted_sum += env["theta"] * w
    total_weight += w
posterior_mean_theta = weighted_sum / total_weight

theta_vals = np.array([env["theta"] for env in samples])
weights_array = np.array(weights)

plt.figure(figsize=(10, 6))

# Plot the weighted histogram of posterior samples
counts, bins, _ = plt.hist(theta_vals, bins=50, density=True, weights=weights_array,
                         alpha=0.6, label='Weighted Posterior')

sigma0 = math.sqrt(tau2)

# Create a range of theta values for plotting the prior PDF
theta_range = np.linspace(mu0 - 4*sigma0, mu0 + 4*sigma0, 500)

# Compute the prior PDF for these theta values
prior_pdf = stats.norm.pdf(theta_range, loc=mu0, scale=sigma0)

# Plot the prior PDF
plt.plot(theta_range, prior_pdf, 'r-', lw=2, label='Prior PDF')

plt.xlabel('θ')
plt.ylabel('Density')
plt.title('Posterior Distribution of θ with Prior Overlay')
plt.legend()
plt.show()

print(f"Approx. posterior mean of theta = {posterior_mean_theta:.3f}")